### **Load Environment variables from .env file**

In [1]:
import json
import os
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
import openai
import tiktoken
from IPython.display import display, HTML, JSON, Markdown
# to avoid error: Could not automatically map gpt-35-turbo to a tokeniser...
tiktoken.model.MODEL_TO_ENCODING["gpt-35-turbo"] = "cl100k_base"
tiktoken.model.MODEL_TO_ENCODING["gpt-35-turbo-instruct"] = "cl100k_base"

In [2]:

load_dotenv()
# env variables that are used by LangChain
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_TYPE'] = "azure"
os.environ['OPENAI_API_VERSION'] = os.getenv("OPENAI_DEPLOYMENT_VERSION")
os.environ['OPENAI_API_BASE'] = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")

OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv(
    "OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

#gpt-35-turbo-instruct
OPENAI_GPT_35_TURBO_INSTRUCT_MODEL_NAME="gpt-35-turbo-instruct"
OPENAI_GPT_35_TURBO_INSTRUCT_DEPLOYMENT_NAME="gpt-35-turbo-instruct"

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_DEPLOYMENT_VERSION")
openai.api_base = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
openai.api_key = os.getenv("OPENAI_API_KEY")

### **Model initialization**

In [3]:

def init_llm(model=OPENAI_GPT_35_TURBO_INSTRUCT_MODEL_NAME,
             deployment_name=OPENAI_GPT_35_TURBO_INSTRUCT_DEPLOYMENT_NAME,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm


### **Prompt engineering techniques**
Those techniques are not specific to summarization, but can be used for any task.

1. Use delimeters to clearly separate the exact text the model should summarize.

    Delimiters could be any kind of punctuation, that separates specific pieces of text. 
    Tripple quotes, Triple backtics, Triple dashes, Angle brackets, XML tags, etc.

2. Ask model for structured output, e.g json, html, etc. If you ask for a json define the structure of the json, e.g. what fields should be in the json.

3. Check assumptions required to do the task.

4. Few-shot prompting. Provide examples of completing tasks and then ask model to perform the task.



In [4]:
#Input text

text = f"""
**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you...

"""


In [5]:
# Use delimiters to clearly separate input text

prompt = f""" Summarize the text delimited by triple backticks into a summary of 20 words. 
```{text}```

Output the summary in the form: Summary: <summary> <|im_end|>
"""

#init model gpt-35-turbo-instruct, which supports Completion API
llm = init_llm()

sum = llm(prompt)
display(Markdown(sum))

Summary: Imaginal Bank offers various investment programs with risk management strategies, including a diversified mutual fund, index tracker ETF, and secure income bond fund. They also provide an online platform for monitoring investments and offer guidance from financial advisors.

In [6]:
#check llm object content. Pay attention that openai.api_resources.completion.Completion
llm

AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='gpt-35-turbo-instruct', temperature=0.0, max_tokens=400, top_p=1.0, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={'stop': ['<|im_end|>']}, openai_api_key='aa87dc8868f34c15b8c299c036c347b6', openai_api_base='https://openailabs303474.openai.azure.com/', openai_organization='', openai_proxy='', batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', tiktoken_model_name=None, deployment_name='gpt-35-turbo-instruct', openai_api_type='azure', openai_api_version='2023-05-15')

In [39]:
# Check the specific conditions/assumptions in the input text

prompt = f""" You will be provided with a text delimited by triple quotes. 
If it contains utterances of a Customer and a Clerk, summarize the text into single sentence.
If the text does not contain utterances of a Customer and a Clerk, 
then just write 'No customer and clerk utterances found'.
\"\"\" {text}  \"\"\"

Output a summary in the form: Summary: <summary> 
<|im_end|>
"""
llm = init_llm()
sum = llm(prompt)
display(Markdown(sum))

Summary: The customer, John, inquires about investment programs at Imaginal Bank and the clerk, Sara, explains the Balanced Growth Fund, Index Tracker ETF, and Secure Income Bond Fund as options for managing risk. The clerk also mentions the Imaginal Investor Dashboard for monitoring investments and offers to schedule an appointment with a financial advisor to assist John in choosing the right program based on his risk tolerance.

#### Give the model time to think. Chain of Thoughts.

Very useful technique is to give the model time to think by specifying the strict steps (sub-tasks) to complete the task and asking for output in a specific format. This is a very powerful technique.


In [40]:
prompt = f""" Your task is to perform the following actions:
1 - Summarize the text delimited by triple backticks into single sentence.
2 - Translate the summary into Portuguese.
3 - List the Customer questions.

Use the following output format:
Summary: <summary>
Translated summary: <portuguese summary>
Customer questions: <enumerated customer questions>

Text: ```{text}``` <|im_end|>
"""

llm = init_llm()
sum = llm(prompt)
display(Markdown(sum))

Summary: The customer, John, is interested in Imaginal Bank's investment programs and asks the clerk, Sara, about them, specifically in terms of risk management. Sara explains the different programs, including the Balanced Growth Fund, Index Tracker ETF, and Secure Income Bond Fund, and mentions that financial advisors are available to help choose the right program based on financial goals and risk tolerance. John also asks about how the Balanced Growth Fund manages risk, and Sara explains the diversification and active management by portfolio managers. John also asks about tools to monitor investments, and Sara mentions the Imaginal Investor Dashboard. Finally, John asks how to get started and Sara offers to schedule an appointment with a financial advisor.

Translated summary: O cliente, John, está interessado nos programas de investimento do Imaginal Bank e pergunta à atendente, Sara, sobre eles, especificamente em termos de gerenciamento de risco. Sara explica os diferentes programas, incluindo o Balanced Growth Fund, Index Tracker ETF e Secure Income Bond Fund, e menciona que os consultores financeiros estão disponíveis para ajudar a escolher o programa certo com base nos objetivos financeiros e tolerância ao risco. John também pergunta sobre como o Balanced Growth Fund gerencia o risco, e Sara explica a diversificação e o gerenciamento ativo pelos gestores de portfólio. John também pergunta sobre ferramentas para monitorar investimentos, e Sara menciona o Imaginal Investor Dashboard. Finalmente, John pergunta como começar e Sara oferece para agendar uma consulta com um consultor financeiro.

Customer questions: 
1. Can you tell me more about Imaginal Bank's investment programs, especially in terms of risk management? 
2. How does the Balanced Growth Fund manage risk? 
3. Does the bank provide any tools to monitor investments? 
4. How can I get started?

##### Let's do some math

In [8]:
prompt = f""" Determine if the student's solution delimited by the triple backticks is correct or not.
Question: When I was 2 years old, my sister was twice my age. I'm now 40 years old. How old is my sister now? 
Student's answer: ```The sister is now 80 years old.``` 
If the student answer is correct, write 'Correct', otherwise write 'Incorrect'. Explain your answer.  
<|im_end|>
"""

llm = init_llm()
sum = llm(prompt)
display(HTML(sum))

##### Ask model to do its own solution and then to compare both solutions and conclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique). 
 Then ask the model to compare its own solution with the solution provided by the model and conclude which is correct.
 Ask model to share it reasoning for the conclusion.

In [9]:
Question = f"""When I was 2 years old my sister was twice of my age . I'm now 40 years old, how old is my sister now?"""
Student_Solution = f""" The sister is now 80 years old."""

prompt = f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
    Reason about every step of your solution and make sure it is correct. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Student's Solution: ```{Student_Solution}```
Question: ```{Question}```


Use the following output format:
Actual solution steps: <your own solution steps>
\nStudent's solution: <student's solution>
\nStudent's solution is correct: <true/false>
<|im_end|>

"""

llm = init_llm()
sum = llm(prompt)
display(Markdown(sum))

Actual solution steps:
1 - First, work out your OWN solution to the problem. 
    Let x be the sister's age when I was 2 years old.
    Then, my age at that time was 2 years old.
    Since my sister was twice my age, we can write the equation:
    x = 2 * 2 = 4
    Therefore, my sister was 4 years old when I was 2 years old.
2 - Second, we can use the fact that I am now 40 years old to find my sister's current age.
    Since I am 40 years old, my sister is 40 - 2 = 38 years older than me.
    Therefore, my sister's current age is 4 + 38 = 42 years old.

Student's solution: The sister is now 80 years old.

Student's solution is correct: False

#### Model Hallucinations and how to avoid them

Hallucination is when the model generates text that is not supported by the input.
To reduce hallucinations, use the techniques listed above, and strictly instruct the model to find the relevant information in the input text. 
If the information is not in the input, instruct the model to generate a specific output, e.g. "I don't know the answer to this question". 

#### Iterative Prompt Development
Prompt engineering is an iterative process. You almost never get the prompt right the first time.
You start with the idea, then you implement the prompt, get the experimental resuslts, do the Error Analysis and iterate again.

**Try -> Analyze -> Clarify Instructions -> Try again**

In more advanced phases refine prompts with a batch of examples.

### Summarization 

In [13]:
# helper function
def summarize_text(llm, prompt_prefix, text_file):

    # read the text file
    with open(text_file, 'r') as file:
        text = file.read()
    # concatenate the prompt with the data
    prompt = prompt_prefix.format(text=text)
    return llm(prompt)

In [14]:
llm = init_llm()
prompt_prefix = """ Summarize the text delimited by triple backticks into 2-3 sentences: ```{text}``` 
<|im_end|>
"""

sum = summarize_text(llm, prompt_prefix,
                     "./data/bank-call-center-transcript.txt")
display(Markdown(sum))

John, a customer of Imaginal Bank, is interested in their investment programs and asks Sara, a customer service representative, about them. Sara explains that they have a Balanced Growth Fund, an Index Tracker ETF, and a Secure Income Bond Fund, each with different levels of risk management. She also mentions their online platform, Imaginal Investor Dashboard, for monitoring investments. John decides to schedule an appointment with a financial advisor to get started.

#### More advanced prompts

In [15]:
llm = init_llm()
prompt_prefix = """ Prepare a summary for the text delimited by the triple backticks```{text}``` based on the points mentioned below. 
Please evaluate whether the clerk successfully accomplished the following tasks:

Greeting the customer politely and professionally.
Accurately understanding the customer's inquiry.
Providing clear and detailed information in response.
Asking questions as needed for clarification.
Discussing both benefits and risks with the customer.
Explaining the tools and resources available to the customer.
Inviting the customer to take further action.
Offering assistance for the next steps.
Ending the conversation on a positive note.

<|im_end|>
"""

sum = summarize_text(llm, prompt_prefix,
                     "./data/bank-call-center-transcript.txt")
display(Markdown(sum))

The clerk successfully accomplished all of the mentioned tasks. They greeted the customer politely and professionally, accurately understood the customer's inquiry, and provided clear and detailed information in response. They also asked questions for clarification and discussed both the benefits and risks of the investment programs. The clerk explained the tools and resources available to the customer and invited them to take further action by scheduling an appointment with a financial advisor. They also offered assistance for the next steps and ended the conversation on a positive note. Overall, the clerk effectively assisted the customer and provided them with the necessary information to make an informed decision about their investments.

##### Output in tabular format

In [21]:
llm = init_llm()
prompt_prefix = """Prepare a summary for the text delimited by the triple backticks:```{text}``` based on the points mentioned below.
Generate the output as a HTML table, with 2 columns: Customer and Clerk.

Assign a color code to each item based on the clerk's performance - 
items that were successfully addressed should be marked in green, 
whereas items that were not met should be highlighted in red.

Here are the evaluation criterias to consider:

Did the clerk greet the customer in a polite and professional manner?
Did the clerk begin the conversation on a negative note?
Did the clerk understand the customer's inquiry?
Did the clerk provide clear and detailed information?
Did the clerk ask questions to clarify the situation?


<|im_end|>
"""

res = summarize_text(llm, prompt_prefix,
                     "./data/bank-call-center-transcript.txt")
display(HTML(res))

##### Output as JSON

In [41]:
llm = init_llm()

prompt_prefix = """" Evaluate the clerk performance from the text delimited by the triple backticks: ```{text}``` based on the points mentioned below.

Did the clerk greet the customer in a polite and professional manner?
Did the clerk comprehend the customer's inquiry accurately?
Did the clerk provide comprehensive and clear information?
Did the clerk ask relevant questions to clarify the customer's situation?
Did the clerk explain the benefits and potential risks to the customer?
Did the clerk detail the tools and resources available to the customer?
Did the clerk encourage the customer to take further action?
Did the clerk offer assistance with proceeding to the next steps?
Did the clerk end the interaction on a positive and upbeat note?"

The output should be presented in JSON format, adhering to the following key-value pairs:

"Greet the Customer Politely and Professionally": "Yes/No",
"Understand the Customer's Inquiry": "Yes/No",
"Provide Clear and Detailed Information": "Yes/No",
"Ask Clarifying Questions": "Yes/No",
"Discuss the Benefits and Risks": "Yes/No",
"Explain Available Tools and Resources": "Yes/No",
"Invite Further Action": "Yes/No",
"Offer to Assist with Next Steps": "Yes/No",
"End on a Positive Note": "Yes/No"

<|im_end|>
"""

res = summarize_text(llm, prompt_prefix,
                     "./data/bank-call-center-transcript.txt")
display(JSON(json.loads(res)))

<IPython.core.display.JSON object>

### Summarize large documents
To summarize larger documents, we can split the document into smaller chunks and summarize each chunk separately. We can then combine the summaries of each chunk to get the final summary.
LangChain has a built-in chain for doing that. 



In [17]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

from langchain.document_loaders import PyPDFLoader
llm = init_llm()

large_pdf_path = "./data/Large_language_model.pdf"
loader = PyPDFLoader(large_pdf_path)

# output type is List[Document]
pages = loader.load_and_split()

# count tokens in the document
total_tokens = 0
encoder = tiktoken.get_encoding("cl100k_base")

for page in pages:
    total_tokens += len(encoder.encode(page.page_content))

# The document has more 13000 tokens. This is too many for the LLM to process in one go.
# This is whay we split the document into smaller chunks.
print(f"Total tokens in the document: {total_tokens}")

Total tokens in the document: 13420


In [12]:
from langchain.chains.summarize import load_summarize_chain

# create summarization chain
# map_reduce summarizes each document on it's own in a "map" step and then "reduce" the summaries into a final summary
summary_chain = load_summarize_chain( 
    llm=llm, chain_type='map_reduce', verbose=False)

##### There are following chain_types: stuff, map_reduce, refine, map-rerank
See here for the details: https://docs.langchain.com/docs/components/chains/index_related_chains

In [13]:
# Many calls in loop causing RateLimitError: Too Many Requests. In this sum = summary_chain.run(pages[0:10]) 
sum = summary_chain.run(pages)

display(Markdown(sum))

 This article discusses various large language models, their capabilities, and their potential applications. It also covers the emergence of new models and techniques, such as instruction fine-tuning and prompting, and the ethical considerations surrounding these models. The article also mentions the upcoming release of several new models and their potential uses in different industries.

In [15]:
#TODO - Upgrade LangChain version to the latest
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain, StuffDocumentsChain

llm = init_llm()

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:
<|im_end|>"""

map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Summary:

<|im_end|>
"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000,
)

In [16]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


In [47]:
map_reduce_chain.run(pages)

'Large language models have become powerful tools in natural language processing, with their capabilities and performance constantly improving. However, concerns about training costs and potential biases remain. Pre-training and fine-tuning techniques are used to optimize these models for specific tasks. Open-source models and collaborations have made these models more accessible, leading to advancements in dialogue and conversational AI. Ethical considerations and responsible AI are also important factors in their development and use. The release of new models, partnerships, and advancements in training data and computing are driving the future of AI and language models, with potential for impact in various industries.'